In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

from preprocessing import preproc
from tools import log_loss_lgbm

from datetime import datetime
import time

path_to_data = "/home/raph/Downloads/"

print("loading data")
training = pd.read_csv(path_to_data+"training.csv")
testing = pd.read_csv(path_to_data+"testing.csv")

loading data


In [25]:
path_to_data = "/home/raph/Downloads/"
to_submit = pd.read_csv(path_to_data+"training.csv")

[('ZnX4ChdxQMVG6LWP4L5HkivSg/PozQTSw8VHAun7cm0=', 1), ('6lSiQuRZwwyYyAIUdhtuChOlERNkLd8tTvU9z2iq7DQ=', 1), ('oyIbYlldwpg0XbBfMRW6NvkKTnkuFONPrwz2xTjD43U=', 1), ('KYSvQvMmpVlF9eo5OJUvJV1+TQvWvbrIH/PcCOu1vYw=', 1), ('0hAhlYaaNVbHUUBsXp62jeuAYc4024xbguIrXI/iQ9Y=', 1), ('Fc+Qjcp+lNGi6WruXYzfCTS9ncnqA1bDRsGoLcMWwfI=', 1), ('Nqw5p5FXezVmKcTN3VRO02844tv+sHkG06fiB5XYsxY=', 1), ('v9m/fnWQrQZQ0eZMx2X+4FORS36u4ismp6D1EvFYF/4=', 1), ('rGnXG/VH8lLJ9XYeQHpT+Hz585zDITHfAEuR2GMVlUA=', 1), ('y1UT4JX2swIe1IWvyZEOM/yhClcp9M8X35h953WymjM=', 1)]


In [2]:
print("changing dates to time stamps")
training["membership_expire_date"] = training.membership_expire_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
training["membership_expire_date"] = training.membership_expire_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

training["transaction_date"] = training.transaction_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
training["transaction_date"] = training.transaction_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

training["registration_init_time"] = training.registration_init_time.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
training["registration_init_time"] = training.registration_init_time.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

testing["membership_expire_date"] = testing.membership_expire_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
testing["membership_expire_date"] = testing.membership_expire_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

testing["transaction_date"] = testing.transaction_date.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
testing["transaction_date"] = testing.transaction_date.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

testing["registration_init_time"] = testing.registration_init_time.apply(lambda x: datetime.strptime(str(x), "%Y-%m-%d").date() if pd.notnull(x) else x)
testing["registration_init_time"] = testing.registration_init_time.apply(lambda x: time.mktime(x.timetuple()) if pd.notnull(x) else 0.0)

changing dates to time stamps


In [3]:
print("preprocessing")
X_train, y_train = preproc(training, mode='train', oneHot=False)
X_test, y_test = preproc(testing, mode="test", oneHot=False)

# parameters
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'binary_logloss'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

i = 0
K = 5
kf = KFold(n_splits=K, random_state=42, shuffle=True)

# training with KFold Cross Validation
results = []
weights = np.zeros(len(y_train))
weights[y_train == 0] = 1
weights[y_train == 1] = 2

preprocessing


In [4]:
print('Start training...')
for train_index, test_index in kf.split(X_train):
    lgb_train = lgb.Dataset(X_train[train_index], y_train[train_index], weight=weights[train_index])
    lgb_eval = lgb.Dataset(X_train[test_index], y_train[test_index], reference=lgb_train, weight=weights[test_index])
    gbm = lgb.train(params,
        train_set=lgb_train,
        num_boost_round=200,
        valid_sets=lgb_eval,
        early_stopping_rounds=10,
        verbose_eval=5,
        feval=log_loss_lgbm)
    res = gbm.predict(X_test)
    i += 1
    results.append(res)

Start training...
Training until validation scores don't improve for 10 rounds.
[5]	valid_0's binary_logloss: 0.355607	valid_0's log loss: 0.242459
[10]	valid_0's binary_logloss: 0.2852	valid_0's log loss: 0.199903
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.472832	valid_0's log loss: 0.311693
Training until validation scores don't improve for 10 rounds.
[5]	valid_0's binary_logloss: 0.35726	valid_0's log loss: 0.243412
[10]	valid_0's binary_logloss: 0.286739	valid_0's log loss: 0.200802
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.475021	valid_0's log loss: 0.31294
Training until validation scores don't improve for 10 rounds.
[5]	valid_0's binary_logloss: 0.35545	valid_0's log loss: 0.242334
[10]	valid_0's binary_logloss: 0.285155	valid_0's log loss: 0.199788
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.472519	valid_0's log loss: 0.311503
Training until validation scores don't improve for 10 rounds.
[5]	valid_0's binar

In [5]:
submission = pd.DataFrame((results[0] + results[1] + results[2] + results[3] + results[4]) / 5)

In [8]:
submission.columns = ["is_churn"]

In [11]:
submission.describe()

,is_churn
count,1.076941e+06
mean,1.083442e-01
std,1.259450e-02
min,1.033879e-01
25%,1.033879e-01
50%,1.034919e-01
75%,1.034919e-01
max,1.513947e-01


In [10]:
submission["msno"] = testing["msno"]

In [12]:
submission.head()

,is_churn,msno
0,0.103388,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=
1,0.103652,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=
2,0.103388,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=
3,0.103388,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=
4,0.103652,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=


In [13]:
submission.to_csv('5Kfold_lgbm.csv', header=True, index=False)
print("created submission file")

created submission file


In [14]:
submission.describe()

,is_churn
count,1.076941e+06
mean,1.083442e-01
std,1.259450e-02
min,1.033879e-01
25%,1.033879e-01
50%,1.034919e-01
75%,1.034919e-01
max,1.513947e-01
